<a href="https://colab.research.google.com/github/greymouse1/spatialanalysis/blob/main/continuity_multiple_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code is licenced under MIT licence.

Author: Nikola G.

Credits:

Tutorial from Momepy package website at http://docs.momepy.org/en/stable/user_guide/graph/coins.html
based on paper by Tripathy et al. (2020)

OpenAI. (2024). ChatGPT (version 4) [Large language model]. OpenAI. https://openai.com/chatgpt



Tripathy, P., Rao, P., Balakrishnan, K., & Malladi, T. (2020). An open-source tool to extract natural continuity and hierarchy of urban street networks. Environment and Planning B: Urban Analytics and City Science. http://dx.doi.org/10.1177/2399808320967680

In [ ]:
!pip install osmnx > /dev/null 2>&1
!pip install momepy > /dev/null 2>&1
!pip install mapclassify>=2.4.0 > /dev/null 2>&1 # install mapclassify with version >=2.4.0
import osmnx as ox
import geopandas as gpd
import momepy
import mapclassify
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
def naturalCities(currentCity,folderPath,cityName):
  # Retrieve the graph within the polygon's boundaries
  # This will pull OSM data from inside the polygon and create a networkX graph

  graph = ox.graph_from_polygon(
      currentCity,
      network_type='drive',  # Choose network type (e.g., 'drive', 'walk', 'bike', etc.)
      simplify=True,         # Simplify graph (remove unnecessary nodes)
      retain_all=False,      # Keep only the largest connected component
      truncate_by_edge=False  # Truncate by edge to keep nodes near the edge
  )

  # Reproject graph
  # Choice of final projection is automatic, original must be WGS84

  bandung_streets = ox.projection.project_graph(graph)

  # Create gdf from graph so it can be used later on

  bandung_gdf = ox.graph_to_gdfs(
      ox.convert.to_undirected(bandung_streets),
      nodes=False,
      edges=True,
      node_geometry=False,
      fill_edge_geometry=True,
  )

  # Calculate continuity from the gdf

  continuity = momepy.COINS(bandung_gdf, angle_threshold=135, flow_mode=False)

  # Pull out stroke

  bandung_stroke_gdf = continuity.stroke_gdf()

  # Save stroke to .shp
  shapefile_path = os.path.join(folderPath, f"polygon_{cityName}.shp")
  bandung_stroke_gdf.to_file(shapefile_path)

  # Pull n_segments out of the gdf
  n_segments = bandung_stroke_gdf['n_segments']

  # Calculate heavy tailed classification
  classifier = mapclassify.HeadTailBreaks(n_segments)

  # Get classification details
  class_intervals = classifier.bins  # Class boundaries
  counts = classifier.counts         # Number of features in each class
  labels = classifier.yb             # Class labels for each feature

  # Save classification output to a text file
  classifier_path = os.path.join(folderPath, f"map_classifier_output_{cityName}.txt")

  with open(classifier_path, "w") as file:
      file.write("HeadTailBreaks Classification\n")
      file.write("================================\n")
      file.write("Class Intervals:\n")
      file.write(", ".join(f"{b:.2f}" for b in class_intervals) + "\n\n")
      file.write("Counts in Each Class:\n")
      file.write(", ".join(str(c) for c in counts) + "\n\n")

  print(f"Classifier output saved to {classifier_path}")

  # Show the classifier (lower and upper bounds plus count)
  print(classifier)

  bandung_stroke_gdf.plot(
      figsize=(15, 15),
      cmap="viridis_r",
      column="n_segments",
      legend=True,
      scheme="headtailbreaks",
  ).set_axis_off()

  output_path_figure = os.path.join(folderPath, f"figure_{cityName}.png")
  plt.savefig(output_path_figure, dpi=600, bbox_inches="tight")

  print(f"Plot for {cityName} saved to {output_path_figure}")

  plt.clf() # clear existing plot and make space for a new one
  # Histogram for n_segments

  bandung_stroke_gdf['n_segments'].plot(kind='hist', bins=40, title='n_segments')
  plt.gca().spines[['top', 'right',]].set_visible(False)
  output_path_histogram = os.path.join(folderPath, f"histogram_{cityName}.png")
  plt.savefig(output_path_histogram, dpi=300, bbox_inches="tight")
  print(f"Histogram for {cityName} saved to {output_path_histogram}")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load shp file with all the polygons
all_pol = gpd.read_file("/content/drive/MyDrive/spatialanalysis/all_pol/all_cities_wgs84.shp")

base_folder = "output_polygons"
os.makedirs(base_folder, exist_ok=True)  # Create base folder if it doesn't exist

# Iterate through each polygon and create a new GeoDataFrame
for index, row in all_pol.iterrows():

    # Define the folder name for the current polygon
    polygon_folder = os.path.join(base_folder, f"polygon_{index}")
    os.makedirs(polygon_folder, exist_ok=True)  # Create folder if it doesn't exist

    # Create a new GeoDataFrame for the current polygon
    new_gdf = gpd.GeoDataFrame(
        [row],
        columns=all_pol.columns,
        crs=all_pol.crs  # Retain the original CRS
    )
    polygon = new_gdf.geometry.iloc[0]  # Extract the Polygon/MultiPolygon geometry

    # Save or process the new GeoDataFrame
    print(f"New GeoDataFrame for polygon at index {index} is loaded")

    naturalCities(polygon,polygon_folder,index)



